In [ ]:
from IPython import display
import math
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

In [ ]:
import nltk

In [277]:
start_date = dt.date(2020, 7, 1)
start=start_date
end_date = dt.date(2021, 8, 13)

number_of_days = end_date-start_date
number_of_days=number_of_days.days
loop=0

RETRY

In [ ]:
import datetime as dt
import pandas as pd
import numpy as np

start_date = dt.date(2021, 5, 15)
start=start_date
end_date = dt.date(2021, 8, 13)


number_of_days = end_date-start_date
number_of_days=number_of_days.days
loop=0

date_list = []
for day in range(number_of_days):
    a_date = (start_date + dt.timedelta(days = day)).isoformat()
    date_list.append(a_date)
    
dates_df = pd.DataFrame(
    {'Date': date_list
    })

import praw
from psaw import PushshiftAPI
import json
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import urllib.parse

import nltk
from pycorenlp import *
import collections
import datetime as dt

api = PushshiftAPI()
delta=dt.timedelta(days=1)
triples_list=[None]*number_of_days

while start_date < (end_date):

    start_epoch=int(pd.Timestamp(start_date).timestamp())
    next_day=start_date+delta
    end_epoch1 =int(pd.Timestamp(next_day).timestamp())
    list1=list(api.search_submissions(after=start_epoch,
                                      before =end_epoch1, 
                                subreddit='worldnews',
                                filter=['title','score']))



    titles=[None]*len(list1)
    scores_all=[None]*len(list1)
    for i in range(0,len(list1)):
        titles[i]=list1[i][4]['title']
        scores_all[i]=list1[i][4]['score']

    daily_news = {'Title':titles,'Score':scores_all}
    daily_news_df = pd.DataFrame(daily_news)
    daily_news_df=daily_news_df.sort_values(by=['Score'], ascending=False)
    daily_news_df=daily_news_df.Title.values.tolist()
    daily_news_df=daily_news_df[0:25]
    new1=daily_news_df
    s=new1
    new=s


    for i in range(0,len(s)):
        txt=s[i]
        s[i]=s[i].replace('%','percent')

    from nltk.corpus import stopwords
    nltk.download('stopwords')
    from nltk.tokenize import word_tokenize

    for i in range(0,len(s)):
        text = s[i]
        text_tokens = word_tokenize(text)

        tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
        new[i]=' '.join(tokens_without_sw)

    import nltk
    nltk.download('averaged_perceptron_tagger')

    for i in range(0,len(s)):   
        sentence = new[i]
        tagged_sentence = nltk.tag.pos_tag(sentence.split())
        edited_sentence = [word for word,tag in tagged_sentence if tag != 'JJ' and tag != 'JJR' and tag != 'JJS' and tag != 'RB' and tag != 'RBR' and tag != 'RBS']
        new1[i]=' '.join(edited_sentence)

    new1=list(filter(lambda a: a != '', new1))

    l = [None] * (len(new1))

    for i in range(0,(len(new1))):
        nlp=StanfordCoreNLP('http://localhost:9000/')
        x=new1[i]
        output = nlp.annotate(x, properties={"annotators":"tokenize,ssplit,pos,depparse,natlog,openie",
                                             "triple.strict":"true","outputFormat": "json",
                                             "openie.max_entailments_per_clause":"1"})

        result = [output["sentences"][0]["openie"] for item in output]
        if len(result[0])>0:
            z=len(result[0])
            relationSent=result[0][z-1]['relation'],result[0][z-1]['subject'],result[0][z-1]['object']
            l[i]=list(relationSent)
        else:
            l[i]=[None,None,None]

    df = pd.DataFrame(l, columns =['P', 'S', 'O'], dtype = float)
    df1 = df.replace(to_replace='None', value=np.nan).dropna()
    triples=df1.values
    all_data=df1.values


    urls=[]


    for z in range(0,len(all_data)):
        try:
            ## initial consts
            BASE_URL = 'http://api.dbpedia-spotlight.org/en/annotate?text={text}&confidence={confidence}&support={support}'
            TEXT = all_data[z][2]
            CONFIDENCE = '0.7'
            SUPPORT = '10'
            REQUEST = BASE_URL.format(
                text=urllib.parse.quote_plus(TEXT), 
                confidence=CONFIDENCE, 
                support=SUPPORT
            )
            HEADERS = {'Accept': 'application/json'}
            sparql = SPARQLWrapper("http://dbpedia.org/sparql")
            all_urls = []

            r = requests.get(url=REQUEST, headers=HEADERS)
            response = r.json()
            resources = response['Resources']
            for res in resources:
                all_urls.append(res['@URI'])

            urls = all_urls + urls

        except:

            pass # doing nothing on exception


    urls = list(dict.fromkeys(urls))
    test_urls=urls


    columns = ['S','P', 'O']
    data = []
    whole_df= pd.DataFrame(data,columns=columns)
    for i in range(0,len(test_urls)):

                line = 'DESCRIBE '
                output_line=line +'<' + test_urls[i] + '>'



                sparql = SPARQLWrapper("http://dbpedia.org/sparql")

                q="""

                    {}

                """.format(output_line)

                sparql.setQuery(q)



                sparql.setReturnFormat(JSON)
                results = sparql.query().convert()



                s_l = [None] * (len(results['results']['bindings']))
                p_l = [None] * (len(results['results']['bindings']))
                o_l = [None] * (len(results['results']['bindings']))
                for i in range(1,(len(results['results']['bindings']))):

                    s_l[i]=results['results']['bindings'][i]['s']['value']
                    p_l[i]=results['results']['bindings'][i]['p']['value']
                    o_l[i]=results['results']['bindings'][i]['o']['value']



                extra_df = pd.DataFrame(list(zip(s_l, p_l,o_l)),
                                   columns =['S', 'P','O'])
                extra_df=extra_df.iloc[1:]

                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageWikiLink']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageExternalLink']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageLength']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageRedirects']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/property/wikiPageUsesTemplate']

                extra_df['O'] = extra_df['O'].astype(str)
                extra_df=extra_df[extra_df['S'].str.contains("http://dbpedia.org/",na=False)]
                extra_df=extra_df[extra_df['P'].str.contains("http://dbpedia.org/",na=False)]
                extra_df=extra_df[extra_df['O'].str.contains("http://dbpedia.org/",na=False)]




                frames=[whole_df,extra_df]

                whole_df=pd.concat(frames)

    date_triples = pd.DataFrame(
        {'Date': [start_date.isoformat()], 'triples':[whole_df.values]
        })
    
    
    triples_list[loop]=whole_df.values
    start_date=start_date+delta
    loop=loop+1

In [251]:
all_triples_dates = pd.DataFrame(
        {'Date': date_list, 'triples_':triples_list
         
        })

In [265]:
day_with_triples=pd.concat([dff1,dff2,all_triples_dates])

In [249]:
#day_with_triples=pd.concat([df11,df22,all_triples_dates])
#dff1
#dff2
#day_with_triples=day_with_triples.dropna()

In [267]:
day_with_triples=day_with_triples.dropna()

In [268]:
#day_with_triples['Date'] = pd.to_datetime(day_with_triples['Date'])
day_with_triples=all_triples_dates
day_with_triples = day_with_triples.set_index(day_with_triples['Date'])
day_with_triples = day_with_triples.sort_index()

In [272]:
day_with_triples

,Date,triples_
Date,,
2020-07-01,2020-07-01,"[[http://dbpedia.org/resource/Ab_Khvor, http:/..."
2020-07-02,2020-07-02,[[http://dbpedia.org/resource/Space_Patrol_(19...
2020-07-03,2020-07-03,"[[http://dbpedia.org/resource/Ditaola_Ditaola,..."
2020-07-04,2020-07-04,"[[http://dbpedia.org/resource/Celia_Viveros, h..."
2020-07-05,2020-07-05,[[http://dbpedia.org/resource/Productivity_Com...
...,...,...
2021-08-08,2021-08-08,[[http://dbpedia.org/resource/1982_bombing_of_...
2021-08-09,2021-08-09,[[http://dbpedia.org/resource/Patience_Torlowe...
2021-08-10,2021-08-10,[[http://dbpedia.org/resource/Emily_Ratajkowsk...


<font size="7">Train model</font>

In [278]:
train = day_with_triples[start.strftime('%Y-%m-%d'):(start+(dt.timedelta(days=number_of_days*0.65))).strftime('%Y-%m-%d')]
valid=day_with_triples[(start+(dt.timedelta(days=number_of_days*0.65))+(dt.timedelta(days=1))).strftime('%Y-%m-%d'):(start+(dt.timedelta(days=number_of_days*0.68))).strftime('%Y-%m-%d')]
test  = day_with_triples[(start+(dt.timedelta(days=number_of_days*0.68))+(dt.timedelta(days=1))).strftime('%Y-%m-%d'):]
print('Train Dataset:',train.shape)
print('Valid Dataset:',valid.shape)
print('Test Dataset:',test.shape)

Train Dataset: (266, 2)
Valid Dataset: (12, 2)
Test Dataset: (130, 2)


In [279]:
arr=train['triples_'][0]
for zz in range(1,len(train['triples_'])):
    if len(train['triples_'][zz])==0:
        continue
    else:
        arr = np.concatenate((arr,train['triples_'][zz]))

In [280]:
val_arr=valid['triples_'][0]
for vv in range(1,len(valid['triples_'])):
    if len(valid['triples_'][vv])==0:
        continue
    else:
        val_arr = np.concatenate((val_arr,valid['triples_'][vv]))

In [281]:
df_of_train = pd.DataFrame(arr, 
             columns=['S', 
                      'P','O'])

df_of_valid = pd.DataFrame(val_arr, 
             columns=['S', 
                      'P','O'])

In [282]:
final_df=df_of_train

In [283]:
valid_final_df=df_of_valid

In [284]:
final_df["S"] = final_df["S"].astype(str)
final_df["P"] = final_df["P"].astype(str)
final_df["O"] = final_df["O"].astype(str)

valid_final_df["S"] = valid_final_df["S"].astype(str)
valid_final_df["P"] = valid_final_df["P"].astype(str)
valid_final_df["O"] = valid_final_df["O"].astype(str)

In [286]:
remove_rel=final_df.groupby('P').count()
remove_rel=remove_rel.sort_values(by=['O'])

single_rel=remove_rel.loc[remove_rel['S'] < 10]
single_rel=single_rel.index.values.tolist()

final_df=final_df[~final_df['P'].isin(single_rel)]

In [287]:
#remove relations in validation set which do not exist in training set

test1=final_df.groupby('P').count()
test1=test1.sort_values(by=['O'])
first_c=test1.iloc[:, 0]
relations_train=first_c.index.values.tolist()

#valid_final_df=valid_final_df[valid_final_df['P'].isin(relations_train)]

In [288]:
test2=final_df.groupby('S').count()
test2=test2.sort_values(by=['O'])
first_c_s=test2.iloc[:, 0]
subject_train=first_c_s.index.values.tolist()
#valid_final_df=valid_final_df[valid_final_df['S'].isin(subject_train)]

In [289]:
test3=final_df.groupby('O').count()
test3=test3.sort_values(by=['S'])
first_c_o=test3.iloc[:, 0]
object_train=first_c_o.index.values.tolist()

In [290]:
valid_final_df=valid_final_df[valid_final_df['S'].isin(subject_train)]
valid_final_df=valid_final_df[valid_final_df['P'].isin(relations_train)]
valid_final_df=valid_final_df[valid_final_df['O'].isin(object_train)]

In [ ]:
#final_df.to_csv('train_data.csv')
#valid_final_df.to_csv('valid_data.csv')
#test.to_csv('test_data.csv')

In [ ]:
import pandas as pd
#final_df=pd.read_csv('train_data.csv')

#final_df=final_df[['S','P','O']]
#final_df["S"] = final_df["S"].astype(str)
#final_df["P"] = final_df["P"].astype(str)
#final_df["O"] = final_df["O"].astype(str)

In [292]:
import ampligraph
ampligraph.latent_features.set_entity_threshold(1658808)

In [293]:
import numpy as np
import ampligraph

from ampligraph.latent_features import TransE

train_all_triple=final_df.values
#valid_all_triple=valid_final_df.values

model = TransE(batches_count=64, seed=0, epochs=30, k=100, eta=20,optimizer='adam', optimizer_params={'lr':0.0001},loss='pairwise', verbose=True)

model.fit(train_all_triple) #, early_stopping=True, early_stopping_params={'x_valid':valid_all_triple,'burn_in':100}

Average TransE Loss:   0.024827: 100%|██████████| 30/30 [4:17:16<00:00, 514.54s/epoch]  


In [ ]:
example_name = 'helloworld.pkl'
ampligraph.utils.save_model(model, model_name_path = example_name)

<font size="7">Embed Daily</font>

In [294]:
import statistics

In [295]:
train['s_overall_av']=None
train['p_overall_av']=None
train['o_overall_av']=None

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [370]:
train

,Date,triples_,s_overall_av,p_overall_av,o_overall_av
Date,,,,,
2020-07-01,2020-07-01,"[[http://dbpedia.org/resource/Ab_Khvor, http:/...",0.000257778,-0.00064785,0.000298753
2020-07-02,2020-07-02,[[http://dbpedia.org/resource/Space_Patrol_(19...,-0.000863363,0.00316943,0.00169336
2020-07-03,2020-07-03,"[[http://dbpedia.org/resource/Ditaola_Ditaola,...",-0.00016974,-0.000199955,-0.000406729
2020-07-04,2020-07-04,"[[http://dbpedia.org/resource/Celia_Viveros, h...",-3.62698e-05,-0.000577759,0.000238436
2020-07-05,2020-07-05,[[http://dbpedia.org/resource/Productivity_Com...,-0.000352343,-0.00062766,-7.58505e-05
...,...,...,...,...,...
2021-03-19,2021-03-19,[],0,0,0
2021-03-20,2021-03-20,[],0,0,0
2021-03-21,2021-03-21,[],0,0,0


In [ ]:
for dayy in range(0,len(train)):
    daily_triple_df = pd.DataFrame(train['triples_'][dayy], 
             columns=['S', 
                      'P','O'])
    remove_rel=df_of_train.groupby('P').count()
    remove_rel=remove_rel.sort_values(by=['O'])

    single_rel=remove_rel.loc[remove_rel['S'] < 10]
    single_rel=single_rel.index.values.tolist()

    daily_triple_df=daily_triple_df[~daily_triple_df['P'].isin(single_rel)]

    daily_triple_df['s_avg_emb']=None
    daily_triple_df['p_avg_emb']=None
    daily_triple_df['o_avg_emb']=None
    daily_triple_df=daily_triple_df.reset_index(drop=True)
    
    if len(daily_triple_df.S.values.tolist())==0:
        train['s_overall_av'][dayy]=0
        train['p_overall_av'][dayy]=0
        train['o_overall_av'][dayy]=0
    else:    
        
        train['s_overall_av'][dayy]=np.average(model.get_embeddings(daily_triple_df.S.values.tolist(),embedding_type='entity'))
        train['p_overall_av'][dayy]=np.average(model.get_embeddings(daily_triple_df.P.values.tolist(),embedding_type='relation'))
        train['o_overall_av'][dayy]=np.average(model.get_embeddings(daily_triple_df.O.values.tolist(),embedding_type='entity'))
    print(train['s_overall_av'][dayy])    

In [ ]:
test['s_overall_av']=None
test['p_overall_av']=None
test['o_overall_av']=None
for dayy2 in range(0,len(test)):   
    daily_triple_df = pd.DataFrame(test['triples_'][dayy2], 
             columns=['S', 
                      'P','O'])


    daily_triple_df=daily_triple_df[daily_triple_df['S'].isin(subject_train)]
    daily_triple_df=daily_triple_df[daily_triple_df['P'].isin(relations_train)]
    daily_triple_df=daily_triple_df[daily_triple_df['O'].isin(object_train)]
    daily_triple_df['s_avg_emb']=None
    daily_triple_df['p_avg_emb']=None
    daily_triple_df['o_avg_emb']=None
    
    if len(daily_triple_df.S.values.tolist())==0:
        test['s_overall_av'][dayy2]=0
        test['p_overall_av'][dayy2]=0
        test['o_overall_av'][dayy2]=0
    else:    
        test['s_overall_av'][dayy2]=np.average(model.get_embeddings(daily_triple_df.S.values.tolist(),embedding_type='entity'))
        test['p_overall_av'][dayy2]=np.average(model.get_embeddings(daily_triple_df.P.values.tolist(),embedding_type='relation'))
        test['o_overall_av'][dayy2]=np.average(model.get_embeddings(daily_triple_df.O.values.tolist(),embedding_type='entity'))
    print(test['o_overall_av'][dayy2])

<font size="7">Add finance data</font>

In [ ]:
import yfinance as yf

# get historical market data
hist = yf.download("AAPL", start=start.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))

date_price = hist[["Close"]]
date_price['Date'] = date_price.index
date_price=date_price[['Date','Close']]
date_price.index = range(0, len(date_price))
#date_price=date_price.set_index('Date', inplace=True)
date_price.index=date_price['Date']

In [361]:
date_price_train=pd.merge(date_price, train, left_index=True, right_index=True)
date_price_train=date_price_train[['triples_','s_overall_av', 'p_overall_av','o_overall_av','Close']]

In [362]:
date_price_test=pd.merge(date_price, test, left_index=True, right_index=True)
date_price_test=date_price_y=date_price_test[['triples_','s_overall_av', 'p_overall_av','o_overall_av','Close']]

In [ ]:
date_price_train['up_down']=None
for p in range(0,len(date_price_train)-1):
    if date_price_train['Close'][p+1]>date_price_train['Close'][p]:
        date_price_train['up_down'][p]=1
    else:
        date_price_train['up_down'][p]=-1

In [ ]:
date_price_test['up_down']=None
for p in range(0,len(date_price_test)-1):
    if date_price_test['Close'][p+1]>date_price_test['Close'][p]:
        date_price_test['up_down'][p]=1
    else:
        date_price_test['up_down'][p]=-1

<font size="7">Prediction</font>

In [365]:
import xgboost as xgb

xgb_cl = xgb.XGBClassifier()

In [366]:
date_price_train=date_price_train.fillna(0)
date_price_test=date_price_test.fillna(0)

In [367]:

X_train = date_price_train.drop(["triples_",'Close','up_down'], axis=1)[:-1]
y_train = date_price_train.up_down[:-1]

X_test= date_price_test.drop(["triples_",'Close','up_down'], axis=1)[:-1]
y_test= date_price_test.up_down[:-1]

In [368]:
np.save('xtrain.npy', X_train)
np.save('ytrain.npy', y_train)
np.save('xtest.npy', X_test)
np.save('ytest.npy', y_test)

In [369]:
from sklearn.metrics import accuracy_score

# Init classifier
xgb_cl = xgb.XGBClassifier()

# Fit
xgb_cl.fit(X_train, y_train)

# Predict
preds = xgb_cl.predict(X_test)

# Score
accuracy_score(y_test, preds)

[13:04:57] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.5604395604395604